In [1]:
import numpy as np
import pandas as pd
import itertools
import copy

from scipy.optimize import fsolve
from scipy.special import gamma

In [2]:
class City(object):
    
    def __init__(self, width, height, hospital_count, subregion_count):
        
        self.width = width
        self.height = height
        #for simplicity, subregion_count = base_count
        
        hospital_x = np.linspace(0, self.width, num = hospital_count + 2)
        self.hospital_location = [(hospital_x[i+1], self.height/2) for i in range(hospital_count)]
        
        self.subregion_count = subregion_count + (subregion_count % 2)
        subregion_x = np.linspace(0, self.width, num = int(self.subregion_count/2 + 2))
        subregion_y = np.linspace(0, self.height, num = 4)
        base_location = [[(subregion_x[i+1], subregion_y[j+1]) for j in range(2)] for i in range(int(self.subregion_count/2))]
        self.base_location = list(itertools.chain(*base_location))

In [3]:
def gen_call_arrival_loc(city):
    return (np.random.uniform(0, city.width),np.random.uniform(0, city.height))

In [4]:
def gen_call_arrival_time():
    return np.random.exponential(1)

In [5]:
def gen_atlocation_service_time():
    return np.random.exponential(12)

In [6]:
def weibull_param_relationship(scale, mean, stddev):
    return stddev**2/mean**2 - gamma(1 + 2/scale)/(gamma(1+1/scale)**2) + 1

def get_weibull_parameters(mean = 30, stddev = 13):
    scale = fsolve(weibull_param_relationship, 1,args=(mean, stddev))
    shape = mean / gamma(1+1/scale)
    
    return scale, shape

def gen_athospital_service_time():
    scale, shape = get_weibull_parameters(mean = 30, stddev = 13)
    return (scale * np.random.weibull(shape))[0]

In [7]:
class Call(object):
    
    def __init__(self, call_index, location, time_arrival, interarrival_time, priority = 0):
        
        self.call_index = call_index
        self.location = location
        
        #initialize the call to be unassigned
        #-1-unassigned, 0 to (N-1) assigned ambulance index 
        self.status = -1
        self.arrival_time = time_arrival
        self.next_arrival_time = self.arrival_time + interarrival_time
        self.priority = priority

In [8]:
def get_distance(location1, location2):
    
    distance = np.abs(location1[0] - location2[0]) + np.abs(location1[1] - location2[1])
    return distance

In [9]:
def get_ambulance_travel_time(distance, speed = 10):
    return distance/speed

In [10]:
class Ambulance(object):
    
    def __init__(self, base):
        
        #fix the ambulance's home base
        self.base = base
        
        #initialize the ambulance idle at home base
        
        #status code: 0-idle at base, 1-going to scene of call, 2-serving at scene of call, 3-going to hospital
        #4-transferring patient at hospital, 5-returning to base
        self.status = 0
        self.origin = base
        self.destination = base # if destination = origin, ambulance is stationary
        self.time = 0
        self.endtime = np.inf
        self.call = -1 # -1 if not assigned to any call
        
    def update_status(self, status, origin, destination, time, endtime):
        self.status = status
        
        self.origin = origin
        self.destination = destination
        self.time = time
        self.endtime = endtime
        
    def redployment(self):
        
        #current heuristic: return to home base
        distance = get_distance(self.origin, self.base)
        travel_time = get_ambulance_travel_time(distance)
        self.update_status(5, self.origin, self.base, copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        self.call = -1
        
    def return_to_base(self):
        self.update_status(0, self.base, self.base, copy.deepcopy(self.endtime), np.inf)
        
        
    def assign_to_call(self, call, time, index):
        
        distance = get_distance(self.origin, call.location)
        travel_time = get_ambulance_travel_time(distance)
        self.update_status(1, self.origin, call.location, time, time + travel_time)
        self.call = call.call_index # updated assigned call index
        
        call.status = index
        
        
    def reach_call_location(self, call, callList):
    
        atlocation_servicetime = gen_atlocation_service_time()
        self.update_status(2, call.location, call.location, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + atlocation_servicetime)

        callList.pop(call.call_index)
    
    def transport_to_hospital(self, city):
        
        #transport to nearest hospital
        
        hospital_list = city.hospital_location
        
        min_distance = city.width + city.height
        nearest_hospital = hospital_list[0]
        for hospital in hospital_list:
            distance = get_distance(self.origin, hospital)
            if distance < min_distance:
                min_distance = distance
                nearest_hospital = hospital
        
        travel_time = get_ambulance_travel_time(min_distance)
        self.update_status(3, self.origin, hospital, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        
    def reach_hospital(self):
        hospital_servicetime = gen_athospital_service_time()
        self.update_status(4, copy.deepcopy(self.destination), copy.deepcopy(self.destination), 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + hospital_servicetime)




In [11]:
def arrival(call_index, ambulanceList, callList, time_arrival, M, timeline):
    
    call = Call(call_index, gen_call_arrival_loc(city), time_arrival, gen_call_arrival_time())
    
    if len(callList) >= M:
        i = timeline.shape[0]
        timeline.loc[i] = [call_index, '', 7, time_arrival]
        print("New call arrived. No more capacity. Reject call.")
    
    else:
    
        print("New call arrived. Add to call list.")
#         callList.append(call)
        callList[call_index] = call

        min_distance = 15
        assign = -1
        index = 0

        nearest_distance = 15
        for ambulance in ambulanceList:

            if ambulance.status == 0:
                distance = get_distance(ambulance.origin, call.location)
                if distance < nearest_distance:
                    nearest_distance = distance
                    assign = index

            index = index + 1

        if assign > -1:
            # when the call arrives, there are ambulances idle at base, so assign the call to the nearest ambulance
            print("Idle ambulance at base available. Assign. Now travelling to call location.")
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, assign, 1, time_arrival]
            ambulanceList[assign].assign_to_call(call, time_arrival, assign)

        else:
            print("No available ambulance at the moment.")
            
    time_arrival = call.next_arrival_time
    
    return ambulanceList, callList, time_arrival, timeline

In [25]:
def get_next_event(time_arrival, ambulanceList, callList, city, M, timeline):
    
    ambulanceEndTime_min = np.inf
    index_min = -1
    index = 0
    for ambulance in ambulanceList:
        if ambulance.endtime < ambulanceEndTime_min:
            ambulanceEndTime_min = copy.deepcopy(ambulance.endtime)
            index_min = index
        
        index = index + 1
    
    next_event_time = min(time_arrival, ambulanceEndTime_min)
    print(time_arrival, ambulanceEndTime_min, next_event_time)
    if next_event_time == time_arrival:
        print("New call arrived.")
        i = timeline.shape[0]
        all_call = set(timeline['Call'])
        call_index = len(all_call) if '' not in all_call else len(all_call)-1
        timeline.loc[i] = [call_index, '', 0, time_arrival]
        ambulanceList, callList, time_arrival, timeline = arrival(call_index, ambulanceList, callList, time_arrival, M, timeline)
        
    else:
        if ambulanceList[index_min].status == 1:
            print("Now reach call location. Start at-location treatment. Remove call from call list.")
            call_index = ambulanceList[index_min].call
            call = callList[call_index]
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 2, next_event_time]
            ambulanceList[index_min].reach_call_location(call, callList)
            
        elif ambulanceList[index_min].status == 2:
            print("Now finish at-location treatment. Start going to hospital.")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 3, next_event_time]
            ambulanceList[index_min].transport_to_hospital(city)
            
        elif ambulanceList[index_min].status == 3:
            print("Now reach hospital. Start transferring patient to hospital.")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 4, next_event_time]
            ambulanceList[index_min].reach_hospital()
            
        elif ambulanceList[index_min].status == 4:
            
            print("Now finish transfering patient to hospital. Decide next step (assign to call or return to base).")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 5, next_event_time]
            
            if len(callList) == 0:
                print("Return to base.")
                ambulanceList[index_min].redployment()
            else:
                print("Call waiting. Assign to the first unassigned call in queue.")
                assign = -1
                call_index = min(callList.keys())
                index = 0
                while assign == -1 and index < M:
                    
                    if call_index not in callList:
                        call_index += 1
                        continue
                    elif callList[call_index].status == -1:
                        assign = call_index
                        
                    call_index += 1
                    index = index + 1
                
                if index == M:
                    # calls in callList have all been assigned with an ambulance
                    ambulanceList[index_min].redployment()
                else:
                    i = timeline.shape[0]
                    timeline.loc[i] = [assign, index_min, 1, next_event_time]
                    print([assign, index_min, 1, next_event_time])
                    ambulanceList[index_min].assign_to_call(callList[assign], next_event_time, index_min)
                    
        elif ambulanceList[index_min].status == 5:
            i = timeline.shape[0]
            timeline.loc[i] = ['', index_min, 6, next_event_time]
            print("Now reployed ambulance reach base. Start idling.")
            ambulanceList[index_min].return_to_base()
    
    return time_arrival, ambulanceList, callList, timeline

In [13]:
def print_ambulance(ambulanceList):
    statusList = []
    originList = []
    destinationList = []
    timeList = []
    endtimeList = []
    
    for ambulance in ambulanceList:
        statusList.append(ambulance.status)
        originList.append(ambulance.origin)
        destinationList.append(ambulance.destination)
        timeList.append(ambulance.time)
        endtimeList.append(ambulance.endtime)
    
    return statusList, originList, destinationList, timeList, endtimeList
    

In [14]:
def print_call(callList):
    
    statusList = []
    locationList = []
    timeList = []
    
    for call_index in callList:
        call = callList[call_index]
        statusList.append(call.status)
        locationList.append(call.location)
        timeList.append(call.arrival_time)
    
    return statusList, locationList, timeList

In [15]:
def get_jobcount(timeline):
    timediff = np.append(np.diff(timeline['Timestamp']), 0)
    timeline['timediff'] = timediff
    
    n = timeline.shape[0]
    numCalls = np.zeros(n)
    
    count = 0
    for i in range(n):
        event = timeline.iloc[i]['Event']
        if event == 0:
            count += 1
        elif event == 5 or event == 7: 
            count -= 1
            
        if count <0:
            print("hi")
            
        numCalls[i] = count
        
    numCalls[-1] = numCalls[n-2]
    timeline['numCalls'] = numCalls
    return timeline

In [16]:
def get_jobs(timeline):
    total = max(timeline['Call'])
    
    arrival = np.zeros(total+1)*np.nan
    assign = np.zeros(total+1)*np.nan
    reach = np.zeros(total+1)*np.nan
    onsite = np.zeros(total+1)*np.nan
    transfer = np.zeros(total+1)*np.nan
    finish = np.zeros(total+1)*np.nan
    
    for i in range(total + 1):
        c = timeline[timeline['Call'] == i]
        n = c.shape[0]
#         print(n)
        for index, row in c.iterrows():
            t = row['Timestamp']
            event = row['Event']
            if event == 0:
                arrival[i] = t
            elif event == 1:
                assign[i] = t if n > 1 else np.nan
            elif event == 2:
                reach[i] = t if n > 2 else np.nan
            elif event == 3:
                onsite[i] = t if n > 3 else np.nan
            elif event == 4:
                transfer[i] = t if n > 4 else np.nan
            elif event == 5:
                finish[i] = t if n > 5 else np.nan
            elif event == 7:
                finish[i] = t
#         print(n, arrival[i], assign[i], reach[i], onsite[i], transfer[i], finish[i])
        
    columns = ['arrival_time', 'assigned_time', 'reach_patient', 'finish_onsite', 'reach_hospital', 'finish']
    data = list(zip(arrival, assign, reach, onsite, transfer, finish))
    df = pd.DataFrame(data, columns=columns)
    
    df['waiting_time'] = df['assigned_time'] - df['arrival_time']
    df['total_time'] = df['finish'] - df['arrival_time']
    return df

In [ ]:
def performance_metric(timeline, df, target):
    P = timeline.groupby('numCalls')['timediff'].sum() / t
    
    expectn = sum(P * P.index)
    expectw = sum(P[c+1:] * (P.index[c+1:]-c))
    utilization = (expectn - expectw) / c
    
    print('\nUtilization:', utilization)
    print('Expected number of jobs in system:', expectn)
    print('Expected number of jobs in queue:', expectw)
    
    df_complete = df.dropna(axis=0)
    print('Expected time in queue:', np.mean(df_complete['waiting_time']))
    print('Expected time in system:', np.mean(df_complete['total_time']))
    
    assigned = df[df['assigned_time'] != np.nan]
    count = 0
    for index, row in assigned.iterrows():
        if np.isnan(row['total_time']) or row['total_time'] > target:
            count += 1
    
    print("Total unreachable calls:", count)
    
    
    
    
    

# Set up the City

In [26]:
city = City(10, 5, 2, 4)
ambulanceList = [Ambulance(city.base_location[0]), Ambulance(city.base_location[1])]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Ambulance', 'Event', 'Timestamp'])

# Repetitively run the following two blocks to visualize the process

In [27]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

print("Next Arrival: ", time_arrival)

print("Ambulance List Print: ")
print("Status List: ", am_statusList)
print("Origin List: ", am_originList)
print("Destination List: ", am_destinationList)
print("Event start time List: ", am_timeList)
print("Event end time List: ", am_endtimeList)

print("Call List Print: ")
print("Status List: ", cl_statusList)
print("Location List: ", cl_locationList)
print("Arrival time List: ", cl_timeList)

0 inf 0
New call arrived.
New call arrived. Add to call list.
Idle ambulance at base available. Assign. Now travelling to call location.
Next Arrival:  0.051650502094161575
Ambulance List Print: 
Status List:  [1, 0]
Origin List:  [(3.3333333333333335, 1.6666666666666667), (3.3333333333333335, 3.3333333333333335)]
Destination List:  [(8.263629227751316, 0.9130362408749165), (3.3333333333333335, 3.3333333333333335)]
Event start time List:  [0, 0]
Event end time List:  [0.5683926320209733, inf]
Call List Print: 
Status List:  [0]
Location List:  [(8.263629227751316, 0.9130362408749165)]
Arrival time List:  [0]


In [28]:
T = 60*24*2

while timeline.iloc[-1]['Timestamp'] < T:
    time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 3, timeline)

0.051650502094161575 0.5683926320209733 0.051650502094161575
New call arrived.
New call arrived. Add to call list.
Idle ambulance at base available. Assign. Now travelling to call location.
0.06246603820468849 0.5683926320209733 0.06246603820468849
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
1.378652000426968 0.5683926320209733 0.5683926320209733
Now reach call location. Start at-location treatment. Remove call from call list.
1.378652000426968 0.5958420688740195 0.5958420688740195
Now reach call location. Start at-location treatment. Remove call from call list.
1.378652000426968 0.6210237577671874 0.6210237577671874
Now finish at-location treatment. Start going to hospital.
1.378652000426968 0.9394163897881607 0.9394163897881607
Now reach hospital. Start transferring patient to hospital.
1.378652000426968 3.498147274721454 1.378652000426968
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
2.0496

38.99853163380109 35.16464024952841 35.16464024952841
Now reach hospital. Start transferring patient to hospital.
38.99853163380109 37.6691365199655 37.6691365199655
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[26, 1, 1, 37.6691365199655]
38.99853163380109 38.1379142029302 38.1379142029302
Now reach call location. Start at-location treatment. Remove call from call list.
38.99853163380109 52.499290668661246 38.99853163380109
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
41.78032199214097 52.499290668661246 41.78032199214097
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
41.810138877133745 52.499290668661246 41.810138877133745
New call arrived.
New call arrived. No more capacity. Reject call.
42.22593226983165 52.499290668661246 42.22593226983165
New call arrived.
New call arrived. No mo

New call arrived. No more capacity. Reject call.
97.55042516665839 96.55718679663721 96.55718679663721
Now reach call location. Start at-location treatment. Remove call from call list.
97.55042516665839 103.0861072965376 97.55042516665839
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
99.51253026537404 103.0861072965376 99.51253026537404
New call arrived.
New call arrived. No more capacity. Reject call.
100.21229187653644 103.0861072965376 100.21229187653644
New call arrived.
New call arrived. No more capacity. Reject call.
101.00202969193235 103.0861072965376 101.00202969193235
New call arrived.
New call arrived. No more capacity. Reject call.
101.41105283830335 103.0861072965376 101.41105283830335
New call arrived.
New call arrived. No more capacity. Reject call.
101.75389801513865 103.0861072965376 101.75389801513865
New call arrived.
New call arrived. No more capacity. Reject call.
105.95849934932251 103.0861072965376 103.0861072965376
N

New call arrived. No more capacity. Reject call.
155.69414641687152 156.21952025103613 155.69414641687152
New call arrived.
New call arrived. No more capacity. Reject call.
156.55329312379 156.21952025103613 156.21952025103613
Now finish at-location treatment. Start going to hospital.
156.55329312379 156.2831907224924 156.2831907224924
Now reach hospital. Start transferring patient to hospital.
156.55329312379 158.70985984899363 156.55329312379
New call arrived.
New call arrived. No more capacity. Reject call.
159.0648301098925 158.70985984899363 158.70985984899363
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[136, 1, 1, 158.70985984899363]
159.0648301098925 159.4353155292354 159.0648301098925
New call arrived.
New call arrived. No more capacity. Reject call.
159.06593080451407 159.4353155292354 159.06593080451407
New call arrived.
New call arrived. No more capacity. Reject c

New call arrived. No more capacity. Reject call.
200.64482363779913 200.88010786454947 200.64482363779913
New call arrived.
New call arrived. No more capacity. Reject call.
201.19572650445696 200.88010786454947 200.88010786454947
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[188, 1, 1, 200.88010786454947]
201.19572650445696 201.22192182247602 201.19572650445696
New call arrived.
New call arrived. No more capacity. Reject call.
201.32697270191645 201.22192182247602 201.22192182247602
Now reach call location. Start at-location treatment. Remove call from call list.
201.32697270191645 212.17637745146686 201.32697270191645
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
201.52851861507375 212.17637745146686 201.52851861507375
New call arrived.
New call arrived. No more capacity. Reject call.
201.66533499187193 212.17637745146686 201.665

...

New call arrived. No more capacity. Reject call.
2696.3534486844183 2701.8600073671196 2696.3534486844183
New call arrived.
New call arrived. No more capacity. Reject call.
2696.4260930241053 2701.8600073671196 2696.4260930241053
New call arrived.
New call arrived. No more capacity. Reject call.
2696.7001671159205 2701.8600073671196 2696.7001671159205
New call arrived.
New call arrived. No more capacity. Reject call.
2697.3980124238424 2701.8600073671196 2697.3980124238424
New call arrived.
New call arrived. No more capacity. Reject call.
2697.821984436951 2701.8600073671196 2697.821984436951
New call arrived.
New call arrived. No more capacity. Reject call.
2698.7560034170006 2701.8600073671196 2698.7560034170006
New call arrived.
New call arrived. No more capacity. Reject call.
2699.9360198029754 2701.8600073671196 2699.9360198029754
New call arrived.
New call arrived. No more capacity. Reject call.
2700.092933605431 2701.8600073671196 2700.092933605431
New call arrived.
New call arr

New call arrived. No more capacity. Reject call.
2735.0033406459006 2734.482885135377 2734.482885135377
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[2737, 1, 1, 2734.482885135377]
2735.0033406459006 2734.5239555197627 2734.5239555197627
Now reach call location. Start at-location treatment. Remove call from call list.
2735.0033406459006 2737.793109329168 2735.0033406459006
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
2735.9712012347327 2737.793109329168 2735.9712012347327
New call arrived.
New call arrived. No more capacity. Reject call.
2736.3466576994974 2737.793109329168 2736.3466576994974
New call arrived.
New call arrived. No more capacity. Reject call.
2737.154294632349 2737.793109329168 2737.154294632349
New call arrived.
New call arrived. No more capacity. Reject call.
2737.6318593455476 2737.793109329168 2737.63185934554

New call arrived. No more capacity. Reject call.
2767.6059320284717 2767.765399375864 2767.6059320284717
New call arrived.
New call arrived. No more capacity. Reject call.
2767.6553102399675 2767.765399375864 2767.6553102399675
New call arrived.
New call arrived. No more capacity. Reject call.
2768.09454296788 2767.765399375864 2767.765399375864
Now finish at-location treatment. Start going to hospital.
2768.09454296788 2767.9308056506093 2767.9308056506093
Now reach hospital. Start transferring patient to hospital.
2768.09454296788 2770.333276606701 2768.09454296788
New call arrived.
New call arrived. No more capacity. Reject call.
2768.884774749518 2770.333276606701 2768.884774749518
New call arrived.
New call arrived. No more capacity. Reject call.
2769.812247606978 2770.333276606701 2769.812247606978
New call arrived.
New call arrived. No more capacity. Reject call.
2770.709624722203 2770.333276606701 2770.333276606701
Now finish transfering patient to hospital. Decide next step (a

New call arrived. No more capacity. Reject call.
2801.163895872865 2799.185631354098 2799.185631354098
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[2828, 1, 1, 2799.185631354098]
2801.163895872865 2799.3492115139043 2799.3492115139043
Now reach call location. Start at-location treatment. Remove call from call list.
2801.163895872865 2801.54429672938 2801.163895872865
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
2801.7155164238534 2801.54429672938 2801.54429672938
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[2829, 0, 1, 2801.54429672938]
2801.7155164238534 2801.8381714241546 2801.7155164238534
New call arrived.
New call arrived. No more capacity. Reject call.
2802.1394280717554 2801.8381714241546 2801.8381714241546
Now

New call arrived. Add to call list.
No available ambulance at the moment.
2842.9315812830405 2851.4006772099983 2842.9315812830405
New call arrived.
New call arrived. No more capacity. Reject call.
2843.5999664512547 2851.4006772099983 2843.5999664512547
New call arrived.
New call arrived. No more capacity. Reject call.
2844.8373679697042 2851.4006772099983 2844.8373679697042
New call arrived.
New call arrived. No more capacity. Reject call.
2845.1101254743044 2851.4006772099983 2845.1101254743044
New call arrived.
New call arrived. No more capacity. Reject call.
2845.1444945296407 2851.4006772099983 2845.1444945296407
New call arrived.
New call arrived. No more capacity. Reject call.
2848.2341759432675 2851.4006772099983 2848.2341759432675
New call arrived.
New call arrived. No more capacity. Reject call.
2848.4880300130303 2851.4006772099983 2848.4880300130303
New call arrived.
New call arrived. No more capacity. Reject call.
2848.5225807198017 2851.4006772099983 2848.5225807198017
N

In [29]:
timeline

,Call,Ambulance,Event,Timestamp
0,0,,0,0
1,0,0,1,0
2,1,,0,0.0516505
3,1,1,1,0.0516505
4,2,,0,0.062466
...,...,...,...,...
7468,2922,,7,2878.88
7469,2923,,0,2879.6
7470,2923,,7,2879.6
7471,2924,,0,2880.47


In [30]:
timeline = get_jobcount(timeline)

In [31]:
timeline

,Call,Ambulance,Event,Timestamp,timediff,numCalls
0,0,,0,0,0,1.0
1,0,0,1,0,0.0516505,1.0
2,1,,0,0.0516505,0,2.0
3,1,1,1,0.0516505,0.0108155,2.0
4,2,,0,0.062466,0.505927,3.0
...,...,...,...,...,...,...
7468,2922,,7,2878.88,0.720009,5.0
7469,2923,,0,2879.6,0,6.0
7470,2923,,7,2879.6,0.869889,5.0
7471,2924,,0,2880.47,0,6.0
